<a href="https://colab.research.google.com/github/shivadar/Sql-w-NEON-and-Colab/blob/main/c226_sql_w_neon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install SQLAlchemy psycopg2-binary pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 23.9 MB/s eta 0:00:00


In [ ]:
# Simplified query cell (uses session-level search_path)
# - Connect with the STUDENT URL
# - Set search_path so we can omit schema prefixes
# - Run example queries and show results

import os, getpass, pandas as pd
from sqlalchemy import create_engine, text

STUDENT_DB_URL = os.environ.get("NEON_STUDENT_DB_URL") or getpass.getpass("Paste STUDENT DB URL (hidden): ")
engine = create_engine(STUDENT_DB_URL, pool_pre_ping=True, future=True)

# Example 1: quick row counts
with engine.connect() as conn:
    counts_sql = """
    SELECT 'product' AS table, COUNT(*) AS rows FROM product
    UNION ALL SELECT 'pc', COUNT(*) FROM pc
    UNION ALL SELECT 'laptop', COUNT(*) FROM laptop
    UNION ALL SELECT 'printer', COUNT(*) FROM printer;
    """
    counts_df = pd.read_sql_query(text(counts_sql), conn)

# Example 2: laptops with screens > 15" (maker, model, screen, price)
with engine.connect() as conn:
    laptops_sql = """
    SELECT p.maker, l.model, l.screen, l.price
    FROM product p
    JOIN laptop l USING (model)
    WHERE l.screen > 15
    ORDER BY l.screen DESC, l.price DESC;
    """
    laptops_df = pd.read_sql_query(text(laptops_sql), conn)

# Display results
print("Row counts:")
display(counts_df)

print("\nLaptops (screen > 15\"):")
display(laptops_df)


Paste STUDENT DB URL (hidden): ··········
Row counts:


,table,rows
0,product,30
1,pc,13
2,laptop,10
3,printer,7



Laptops (screen > 15"):


,maker,model,screen,price
0,E,2001,20.1,3673
1,A,2005,17.0,2500
2,E,2002,17.0,949
3,E,2003,16.4,549
4,G,2010,15.4,2300
5,A,2006,15.4,1700
6,F,2008,15.4,900


In [ ]:
# Example 3: quick row counts
with engine.connect() as conn:
    counts_sql = """
select model from pc where speed >= 3.00;
    """
    result = pd.read_sql_query(text(counts_sql), conn)

# Display results
print("Results:")
display(result)



Results:


,model
0,1005
1,1006
2,1013


In [ ]:
print("Query 1: Get the model, speed, and hard drive size of all PCs with a price less than $1000.")
with engine.connect() as conn:
    query1_sql = """
    SELECT model, speed, hd FROM pc WHERE price < 1000;
    """
    query1_result = pd.read_sql_query(text(query1_sql), conn)
    display(query1_result)

Query 1: Get the model, speed, and hard drive size of all PCs with a price less than $1000.


,model,speed,hd
0,1002,2.10,250
1,1003,1.42,80
2,1004,2.80,250
3,1005,3.20,250
4,1007,2.20,250
5,1008,2.20,250
6,1009,2.00,250
7,1010,2.80,300
8,1011,1.86,160
9,1012,2.80,160


In [ ]:
print("\nQuery 2: Find all makers that produce both PCs and printers.")
with engine.connect() as conn:
    query2_sql = """
    SELECT maker
    FROM product
    WHERE type = 'pc'
    INTERSECT
    SELECT maker
    FROM product
    WHERE type = 'printer';
    """
    query2_result = pd.read_sql_query(text(query2_sql), conn)
    display(query2_result)


Query 2: Find all makers that produce both PCs and printers.


,maker
0,E
1,D


In [ ]:
print("\nQuery 3: List all laptop models and their prices, ordered from most expensive to least expensive.")
with engine.connect() as conn:
    query3_sql = """
    SELECT model, price FROM laptop ORDER BY price DESC;
    """
    query3_result = pd.read_sql_query(text(query3_sql), conn)
    display(query3_result)


Query 3: List all laptop models and their prices, ordered from most expensive to least expensive.


,model,price
0,2001,3673
1,2005,2500
2,2010,2300
3,2006,1700
4,2007,1429
5,2004,1150
6,2002,949
7,2008,900
8,2009,680
9,2003,549
